In [41]:
import setuptools.dist
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import mne
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
import random


def initFilesNames():
    filesNames = []
    for x in range(100, 1000, 100):
        filesNames.append("B0" + str(x + 3) + "T")
    return filesNames


TRAIN_PATH = "../data/raw/train/"
TRAIN_CLEAN_PATH = "../data/raw/train_clean/"
TRAIN_ONLY_PATH = "../data/raw/y_train_only/"
TRAIN_EPOCK = "../data/raw/epock/"
DATA_FILES = initFilesNames()

FILTERS =  [{ "name": "std" ,"l_cut": 0.1, "h_cut": 20 },]

SAMPLING_FREQ = 250
STICHANNEL = "STI101"
CHANNELS_LIST = ["C3", "Cz", "C4", STICHANNEL]
CH_TYPES=["eeg", "eeg", "eeg", "stim"]
ROI = ["C3", "Cz", "C4"]
REFERENCE = ["average"]
MONTAGE = mne.channels.make_standard_montage("standard_1020")

In [2]:
def file_preprocessing(
    fileName, tmin=0, tmax=3.5, lFilter=0, hFilter=30, reference="average"
):
    df_mne = pd.read_csv(TRAIN_PATH + fileName + ".csv")
    df_event_type = pd.read_csv(TRAIN_ONLY_PATH + fileName + ".csv")
    df_event_start = df_mne.loc[(df_mne.EventStart == 1), ["time"]]
    df_event_start = df_event_start.reset_index(drop=True)
    df_event_start_with_type = pd.merge(
        df_event_start, df_event_type, left_index=True, right_index=True
    )
    df_mne = pd.merge(df_mne, df_event_start_with_type, how="left", on=["time", "time"])
    df_mne.drop(
        ["EOG:ch01", "EOG:ch02", "EOG:ch03", "EventStart"], axis=1, inplace=True
    )
    df_mne = df_mne.rename(columns={"EventType": STICHANNEL})
    df_mne.C3 = df_mne.C3 / 1000000
    df_mne.Cz = df_mne.Cz / 1000000
    df_mne.C4 = df_mne.C4 / 1000000
    df_mne.replace({STICHANNEL: 1}, 2, inplace=True)
    df_mne.replace({STICHANNEL: 0}, 1, inplace=True)
    df_mne.fillna({STICHANNEL: 0}, inplace=True)
    data = pd.DataFrame.to_numpy(df_mne[CHANNELS_LIST].transpose(), dtype=np.float64)
    info = mne.create_info(
        ch_names=CHANNELS_LIST,
        sfreq=SAMPLING_FREQ,
        ch_types=CH_TYPES,
    )
    raw = mne.io.RawArray(data, info)
    raw.set_montage(MONTAGE)
    events = mne.find_events(raw, stim_channel=STICHANNEL, consecutive=False)
    mapping = {1: "left", 2: "right"}
    annot_from_events = mne.annotations_from_events(
        events=events,
        event_desc=mapping,
        sfreq=raw.info["sfreq"],
        orig_time=raw.info["meas_date"],
    )
    raw.set_annotations(annot_from_events)
    raw_filt = raw .filter(l_freq=lFilter, h_freq=hFilter)
    events = mne.find_events(raw_filt, stim_channel=CHANNELS_LIST, consecutive=False)
    epochs_evoked = mne.Epochs(
        raw_filt, events, tmin=tmin, tmax=tmax, baseline=None, preload=True
    )

    conditions = ["1", "2"]
    evoked = {c: epochs_evoked.__getitem__(c).average() for c in conditions}

    evokeds_avgref = {
        c: evoked[c].set_eeg_reference(ref_channels=reference)
        for c in evoked.keys()
    }
    
    for condition in evokeds_avgref.keys():
        evokeds_avgref[condition].comment = condition
    return evokeds_avgref

In [3]:
def prepare_train_test(evokeds):
        data_dict = {
        "file": np.array([]),
        "sample": np.array([]),
        "values": np.array([],[],[]),
        "eventType": np.array([]),
        }
        print("**************************************************************")
        print(evokeds[c].count)
        for c in evokeds.keys():
                idx = 0
                for e in evokeds[c]:
                        d = e.get_data()
                        for sidx, s in enumerate(ROI):
                                data_dict["file"] = np.append(data_dict["file"], idx)
                                data_dict["sample"] = np.append(data_dict["sample"], n)
                                data_dict["value"] = np.append(data_dict["values"][sidx], d[sidx])
                                data_dict["eventType"] = np.append(data_dict["eventType"], c)
                                n += 1
                        n = 0
                        idx += 1
    
        df = pd.DataFrame(data_dict)
        target = df['eventType']
        df.drop('eventType', axis=1, inplace= True)
        return train_test_split(df, target, test_size=0.2, random_state=12)

In [26]:
def apply_model(model, title, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return {
        "title": title,
        "score": accuracy_score(y_test, y_pred, normalize=True),
        "confusion": confusion_matrix(y_test, y_pred),
        "classification_report": classification_report(y_test, y_pred),
    }


def apply_models(X_train, X_test, y_train, y_test, report_entries):
    report_entries.append(
        apply_model(
            LogisticRegression(max_iter=1000),
            "Logistic Regression",
            X_train,
            y_train,
            X_test,
            y_test,
        )
    )

    adaboost = GridSearchCV(
        AdaBoostClassifier(),
        param_grid={
            "n_estimators": range(30, 101, 10),
            "learning_rate": [1, 0.1, 0.01],
        },
    )
    adaboost.fit(X_train, y_train)
    bm = adaboost.best_estimator_
    report_entries.append(
        apply_model(
            bm,
            "AdaBoost",
            X_train,
            y_train,
            X_test,
            y_test,
        )
    )

    report_entries.append(
        apply_model(
            KNeighborsClassifier(n_neighbors=3),
            "KNeighborsClassifier k: 3",
            X_train,
            y_train,
            X_test,
            y_test,
        )
    )

    report_entries.append(
        apply_model(
            RandomForestClassifier(criterion="entropy", max_depth=3),
            "RandomForestClassifier",
            X_train,
            y_train,
            X_test,
            y_test,
        )
    )
    report_entries.append(
        apply_model(
            svm.SVC(gamma=0.01, kernel="poly"), "svm", X_train, y_train, X_test, y_test
        )
    )


def print_report(report):
    for r in report:
        print(
            "------------------------------------------------------------------------------------------"
        )
        print(
            "min:",
            r["range"]["min"],
            "max: ",
            r["range"]["max"],
            "lowCut: ",
            r["filter"]["low"],
            "hightCut: ",
            r["filter"]["hight"],
            "reference: ",
            r["reference"],
        )
        print(
            "------------------------------------------------------------------------------------------"
        )
        for e in r["entries"]:
            print(e["title"], "score: ", e["score"])
            print(e["confusion"])
            print(e["classification_report"])

In [5]:
def my_train_test_split(df, test_size=0.2, random_state=42):
    random.seed(random_state)
    nb = round(len(df['file'].unique()) * test_size)
    X_test = []
    X_train = []
    for i in range(1, 3):
        idx = random.sample(range(0, len(df['file'].unique())), nb)
        X_test.append(df[(df['file'].isin(idx)) & (df['eventType']==i)])
        X_train.append(df[(~df['file'].isin(idx)) & (df['eventType']==i)])
    
    X_test = pd.concat([X_test[0], X_test[1]])
    X_train = pd.concat([X_train[0], X_train[1]])
    y_test = X_test['eventType']
    y_train = X_train['eventType']
    X_test.drop(["eventType"], axis=1, inplace=True)
    X_train.drop(["eventType"], axis=1, inplace=True)
    
    X_train.to_csv(TRAIN_EPOCK + "Xtrain.csv", index_label='index')
    y_train.to_csv(TRAIN_EPOCK + "ytrain.csv", index_label='index')
    X_test.to_csv(TRAIN_EPOCK + "Xtest.csv", index_label='index')
    y_test.to_csv(TRAIN_EPOCK + "ytest.csv", index_label='index')
    
    return X_train, X_test, y_train, y_test
  


In [70]:
from sklearn.preprocessing import RobustScaler


def prepare_train_test(evokeds):
    data_dict = {
        "file": np.array([]),
        "sample": np.array([]),
        ROI[0]: np.array([]),
        ROI[1]: np.array([]),
        ROI[2]: np.array([]),
        "psdalpha" + ROI[0]: np.array([]),
        "psdalpha" + ROI[1]: np.array([]),
        "psdalpha" + ROI[2]: np.array([]),
        "psdsigma" + ROI[0]: np.array([]),
        "psdsigma" + ROI[1]: np.array([]),
        "psdsigma" + ROI[2]: np.array([]),
        "eventType": np.array([]),
    }

    FREQ_BANDS = {
        "theta": [4.5, 8.5],
        "alpha": [8.5, 11.5],
        "sigma": [11.5, 15.5],
        "beta": [15.5, 30],
    }

    for c in evokeds.keys():
        idx = 0
        n = 0
        for e in evokeds[c]:
            d = e.get_data()
            psdalphasum = [0, 0, 0]
            psdsigmasum = [0, 0, 0]
            for sidx, s in enumerate(ROI):
                psdalpha = e.compute_psd(
                    fmin=FREQ_BANDS["alpha"][0], fmax=FREQ_BANDS["alpha"][1]
                )[s]
                psdalphasum[sidx] = psdalpha.sum()
                psdsigma = e.compute_psd(
                    fmin=FREQ_BANDS["sigma"][0], fmax=FREQ_BANDS["sigma"][1]
                )[s]
                psdsigmasum[sidx] = psdsigma.sum()

            for sidx, s in enumerate(ROI):
                for vidx, v in enumerate(d[sidx]):
                    if sidx == 0:
                        data_dict["file"] = np.append(data_dict["file"], idx)
                        data_dict["sample"] = np.append(data_dict["sample"], n)
                        data_dict["eventType"] = np.append(data_dict["eventType"], c)
                    n += 1
                    data_dict["psdalpha" + s] = np.append(
                        data_dict["psdalpha" + s], psdalphasum[sidx]
                    )
                    data_dict["psdsigma" + s] = np.append(
                        data_dict["psdsigma" + s], psdsigmasum[sidx]
                    )
                    data_dict[s] = np.append(data_dict[s], v)
            n = 0
            idx += 1
    df = pd.DataFrame(data_dict)
    df = df.astype({"file": int, "sample": int, "eventType": int})
    scaler = RobustScaler()
    scaler.fit(
        df[
            [
                ROI[0],
                ROI[1],
                ROI[2],
                "psdalpha" + ROI[0],
                "psdalpha" + ROI[1],
                "psdalpha" + ROI[2],
                "psdsigma" + ROI[0],
                "psdsigma" + ROI[1],
                "psdsigma" + ROI[2],
            ]
        ]
    )
    df[
        [
            ROI[0],
            ROI[1],
            ROI[2],
            "psdalpha" + ROI[0],
            "psdalpha" + ROI[1],
            "psdalpha" + ROI[2],
            "psdsigma" + ROI[0],
            "psdsigma" + ROI[1],
            "psdsigma" + ROI[2],
        ]
    ] = scaler.transform(
        df[
            [
                ROI[0],
                ROI[1],
                ROI[2],
                "psdalpha" + ROI[0],
                "psdalpha" + ROI[1],
                "psdalpha" + ROI[2],
                "psdsigma" + ROI[0],
                "psdsigma" + ROI[1],
                "psdsigma" + ROI[2],
            ]
        ]
    )
    df[ROI[0] + "-" + ROI[2]] = df[ROI[0]] - df[ROI[2]]

    df["psdalpha" + ROI[0] + "-" + "psdalpha" + ROI[2]] = (
        df["psdalpha" + ROI[0]] - df["psdalpha" + ROI[2]]
    )
    df["psdsigma" + ROI[0] + "-" + "psdsigma" + ROI[2]] = (
        df["psdsigma" + ROI[0]] - df["psdsigma" + ROI[2]]
    )

    df.to_csv(TRAIN_EPOCK + "test.csv", index_label="index")
    return my_train_test_split(df, test_size=0.2)


ranges = [
    {"min": 0.40, "max": 0.60},
]

report = []
for r in ranges:
    print(
        "***********************************************************************************************"
    )
    for f in FILTERS:
        for ref in REFERENCE:
            evokeds = {"1": [], "2": []}
            for d in DATA_FILES:
                evokedFileContent = file_preprocessing(
                    d,
                    tmin=r["min"],
                    tmax=r["max"],
                    lFilter=f["l_cut"],
                    hFilter=f["h_cut"],
                    reference=ref,
                )
                for c in evokedFileContent.keys():
                    evokeds[c].append(evokedFileContent[c])
            X_train, X_test, y_train, y_test = prepare_train_test(evokeds)
            report_entries = []
            apply_models(X_train, X_test, y_train, y_test, report_entries)
            report.append(
                {
                    "range": {"min": r["min"], "max": r["max"]},
                    "filter": {"low": f["l_cut"], "hight": f["h_cut"]},
                    "reference": r,
                    "entries": report_entries,
                }
            )
print_report(report)

***********************************************************************************************
Creating RawArray with float64 data, n_channels=4, n_times=469011
    Range : 0 ... 469010 =      0.000 ...  1876.040 secs
Ready.
['STI101']
160 events found on stim channel STI101
Event IDs: [1 2]
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 8251 samples (33.004 s)

['C3', 'Cz', 'C4', 'STI101']
160 events found on stim channel STI101
Event IDs: [1 2]
Not setting metadata
160 matching events found
No baseli

d:\perso\Informatique\Formations\ProjetData\mar24_cds_eeg\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
d:\perso\Informatique\Formations\ProjetData\mar24_cds_eeg\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
d:\perso\Informatique\Formations\ProjetData\mar24_cds_eeg\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
d:\perso\Informatique\Formations\ProjetData\mar24_cds_eeg\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The

------------------------------------------------------------------------------------------
min: 0.4 max:  0.6 lowCut:  0.1 hightCut:  20 reference:  {'min': 0.4, 'max': 0.6}
------------------------------------------------------------------------------------------
Logistic Regression score:  0.7303921568627451
[[102   0]
 [ 55  47]]
              precision    recall  f1-score   support

           1       0.65      1.00      0.79       102
           2       1.00      0.46      0.63       102

    accuracy                           0.73       204
   macro avg       0.82      0.73      0.71       204
weighted avg       0.82      0.73      0.71       204

AdaBoost score:  0.6519607843137255
[[66 36]
 [35 67]]
              precision    recall  f1-score   support

           1       0.65      0.65      0.65       102
           2       0.65      0.66      0.65       102

    accuracy                           0.65       204
   macro avg       0.65      0.65      0.65       204
weighted av